In [1]:
import matplotlib as mpl
mpl.use("pgf")
import numpy as np
pgf_with_pdflatex = {
    "pgf.texsystem": "pdflatex",
    "pgf.preamble": [
         r"\usepackage[utf8x]{inputenc}",
         r"\usepackage[T1]{fontenc}",
         r"\usepackage{mathptmx}",
         r"\usepackage{mathtools}",
#          r"\usepackage{biblatex}",
         ],
    "text.usetex": True,
    "figure.figsize": [4, 3.5],
    "axes.labelsize": 12,
    "font.size": 12,
    "font.family": "serif",
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "lines.linewidth": 1.5,
    "lines.linestyle": None,
    "lines.markeredgewidth": 0.15,
    "xtick.major.size": 5,
    "xtick.major.width": 1,
    "xtick.minor.size": 2.5,
    "xtick.minor.width": 1,
    "ytick.major.size": 5,
    "ytick.major.width": 1,
    "ytick.minor.size": 2.5,
    "ytick.minor.width": 1,
}
mpl.rcParams.update(pgf_with_pdflatex)

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, AutoMinorLocator
from matplotlib.lines import Line2D

In [2]:
markers = ['o', 's', 'd', '^', '*', '+', 'v', '<', '>', 'x']
def mfunc(syms):
    while True:
        for s in syms:
            yield s

In [3]:
df = pd.read_csv('ignition-delay/dme-meoh-data.csv', header=0, index_col=[0,1])
df = df.reindex(['100% DME', '75% DME', '50% DME', '25% DME', '0% DME'], level=0)
df['Percent'] = df['Error']/(2*df['Tig'])*100

In [4]:
sim_df = pd.read_csv('simulation-comparison/output.csv')
sims = {'100:0': '100% DME', '75:25': '75% DME', '50:50': '50% DME', '25:75': '25% DME', '0:100': '0% DME'}
sim_df.rename(columns=sims, inplace=True)

In [5]:
fig, ax_ign_delay = plt.subplots()
ax_ign_delay.set_yscale('log')
ax_ign_delay.set_ylabel('Ignition Delay, ms')
ax_ign_delay.set_xlabel('$1000/T_C$, 1/K')

plot_opts = {'markersize': 6, 'fmt': '', 'elinewidth': 1.0}

mark = mfunc(markers)

for c, new_df in df[(df['Tig'] < 100) & (df['Tig'] > 5) & (df['Percent'] < 10)].groupby(level=0):
    color = next(ax_ign_delay._get_lines.prop_cycler)['color']
    ax_ign_delay.errorbar(new_df['1000/Tc'], new_df['Tig'], yerr=np.array(new_df['Error']), label=c.replace('%', r'\%'), marker=next(mark), color=color, **plot_opts)
    beg = new_df['1000/Tc'][0] + 0.05
    end = new_df['1000/Tc'][-1] - 0.05
    X = sim_df.loc[((sim_df['1000/T'] < beg) & (sim_df['1000/T'] > end)), '1000/T']
    Y = sim_df.loc[((sim_df['1000/T'] < beg) & (sim_df['1000/T'] > end)), c]
    ax_ign_delay.plot(X, Y*1000, linestyle='-', label='', color=color)

# Set y limits
ax_ign_delay.set_ylim(1, 150)

# Set the formatting of the y tick labels
ax_ign_delay.yaxis.set_major_formatter(FormatStrFormatter('%d'))
ax_ign_delay.xaxis.set_minor_locator(AutoMinorLocator(4))

# Set x limits
ax_ign_delay.set_xlim(1.1, 1.7)

# Create the legend, removing the error bars
handles, labels = ax_ign_delay.get_legend_handles_labels()
hnd = []
for i, h in enumerate(handles):
    hnd.append(Line2D([], [], color=h[0].get_color(), marker=markers[i], linestyle='-'))

ax_ign_delay.legend(hnd, labels, loc='lower center', bbox_to_anchor=(0.32, 0.), numpoints=1, frameon=True)

# Create the temperature axes on the top
def convert_inv_temp(temps):
    """Convert a list of temperatures to inverse temperature"""
    return [1000.0/temp for temp in temps]

ax_ign_temp = ax_ign_delay.twiny()

# Set the major tick marks in the temperature scale and convert to invegrse scale
major_temps = np.arange(1000, 500, -100)
major_ticks = convert_inv_temp(major_temps)

# Set the interval for the minor ticks and compute the minor ticks
minor_interval = 20
minor_ticks = []
for maj in major_temps:
    minor_ticks.extend(convert_inv_temp([maj - i*minor_interval for i in range(5)]))
    
# Set the ticks on the axis. Note that the limit setting must be present and must be after setting the ticks
# so that the scale is correct
ax_ign_temp.set_xticks(major_ticks)
ax_ign_temp.set_xticks(minor_ticks, minor=True)
ax_ign_temp.set_xticklabels(['{:d} K'.format(temp) for temp in major_temps])
ax_ign_temp.set_xlim(ax_ign_delay.get_xlim());

In [6]:
fig.savefig('simulation-comparison.pgf', bbox_inches='tight')